In [2]:
#Queensland Shore Pollution 2002-2020

In [3]:
import pandas as pd
import numpy as np

In [4]:
#Import Data

#NB: Original files should be in a "data" folder placed in project's directory of the current notebook

In [5]:
df_0216 = pd.read_csv('data/marinepollutiondata2002to2016.csv', encoding = 'ISO-8859-1')
df_1617 = pd.read_csv('data/marinepollutiondata2016to2017.csv', sep=',', encoding="utf8")
df_1718 = pd.read_excel('data/marinepollutiondata2017to2018.xlsx')
df_1819 = pd.read_csv('data/marinepollutiondata2018to2019.csv', sep=',', encoding="utf8")
df_1920 = pd.read_csv('data/marinepollutiondata2019to2020.csv', sep=',', encoding="utf8")

In [6]:
#Compare headers and details from all files 
#df_0216.describe()
#df_1617.describe()
#df_1718.describe()
#df_1819.describe()
#df_1920.describe()

In [4]:
#Rename column Estimated Litres in the 2016-2017 dataset
df_1617.rename(columns={"Estimated Liters": "Estimated Litres"}, inplace=True)
df_1617

NameError: name 'df_1617' is not defined

In [8]:
# df_1718.describe() shows that the datetype of this dataset is different from the other sets (and will no longer be supported in future Pandas version)
# Convert 'Date' column to DD/MM/YYYY format

df_1718['Date'] = df_1718['Date'].dt.strftime('%d/%m/%Y')
df_1718.describe()

,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
count,49,49,42,39,49,49,49,39
unique,45,5,3,5,3,47,18,15
top,28/10/2017,Brisbane,Ship,Unknown,Port Limits,"Gladstone Marina, Gladstone",Diesel,Unknown
freq,3,19,21,20,30,3,18,10


In [9]:
#Merge our datasets in a single Dataframe, and reset the original index

years = [df_0216, df_1617, df_1718, df_1819, df_1920]

df_pol = pd.concat((years), ignore_index=True)

In [10]:
df_pol.head()


,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
0,2/07/2002,Cairns,Ship,Commercial,Port,"Trinity Inlet, Cairns Harbour",Diesel,Unknown
1,3/07/2002,Cairns,Unknown,Recreational,Coastal Waters,"Marina Mirage, Port Douglas",Bilge,Unknown
2,12/07/2002,Gladstone,Ship,Unknown,Coastal Waters,SE of Double Island Point,Bilge,Unknown
3,22/07/2002,Brisbane,Unknown,Unknown,Coastal Waters,"Town Reach, Brisbane River",Diesel,Unknown
4,1/08/2002,Gold Coast,Ship,Recreational,Coastal Waters,Canal area - Benowa,Diesel,Unknown


In [11]:
df_pol.tail()

,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
1093,15/06/2020,Gladstone,Ship,Recreational,Port Limits,"Gladstone Marina, Gladstone",Other,<90 litres
1094,16/06/2020,Townsville,Ship,Commercial,Port Limits,"Breakwater Terminal, Ross Creek, Townsville",Sewage,> 100 Litres
1095,16/06/2020,Brisbane,Ship,Commercial,Port Limits,"Fisherman Island 3 berth, Port of Brisbane",Hydraulic Oil,100- 200ml
1096,23/06/2020,Cairns,Unknown,Unknown,Port Limits,"Cairns Marlin Marina, Cairns",Diesel,3-5 litres
1097,29/06/2020,Cairns,Land,Recreational,Port Limits,"Cairns Marlin Marina Sail fish Quays , Cairns",Diesel,2-3 litres


In [12]:
df_pol.describe()

,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
count,1098,1098,1091,856,1098,1098,1096,1077
unique,983,14,9,24,13,866,107,160
top,1/02/2009,Brisbane,Ship,Commercial,Coastal Waters,"Marina Mirage, Port Douglas",Diesel,Unknown
freq,3,344,659,273,325,15,463,601


In [13]:
df_pol.isnull().sum()

Date                  0
Region                0
Source                7
Ship Type           242
Area                  0
Location              0
Pollutant             2
Estimated Litres     21
dtype: int64

In [14]:
#Rename the index column to make it our unique key identifying pollution events
#Event ID should start with 1 instead of 0

df_pol.index += 1
df_pol.index = df_pol.index.set_names(['Event ID'])

In [15]:
df_pol.head()

,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
Event ID,,,,,,,,
1,2/07/2002,Cairns,Ship,Commercial,Port,"Trinity Inlet, Cairns Harbour",Diesel,Unknown
2,3/07/2002,Cairns,Unknown,Recreational,Coastal Waters,"Marina Mirage, Port Douglas",Bilge,Unknown
3,12/07/2002,Gladstone,Ship,Unknown,Coastal Waters,SE of Double Island Point,Bilge,Unknown
4,22/07/2002,Brisbane,Unknown,Unknown,Coastal Waters,"Town Reach, Brisbane River",Diesel,Unknown
5,1/08/2002,Gold Coast,Ship,Recreational,Coastal Waters,Canal area - Benowa,Diesel,Unknown


In [16]:
#CLEANING DATA

In [17]:
#get values to lowercase

In [18]:
columns_to_lowercase = ["Source", "Ship Type", "Area", "Pollutant", "Estimated Litres"]
for col in columns_to_lowercase:
    df_pol[col] = df_pol[col].str.lower()

In [19]:
# Convert 'Date' column to datetime format to make it usable in SQL

df_pol['Date'] = pd.to_datetime(df_pol['Date'], format='%d/%m/%Y', errors='coerce')


# Spot the invalid date values
mask = df_pol['Date'].isnull()

# display rows where date is not valid
print(df_pol[mask])

#This confirms our dates are now valid

         Date    Region Source Ship Type         Area  \
Event ID                                                
1005      NaT  Brisbane   land       NaN  port limits   
1009      NaT    Mackay   ship      comm  port limits   

                                                 Location Pollutant  \
Event ID                                                              
1005                       Caltex Wharf, Lytton, Brisbane    diesel   
1009      Maritime Terminal, Port of Airlie, Airlie Beach     sheen   

         Estimated Litres  
Event ID                   
1005               50-60m  
1009            0.5m x 3m  


In [20]:
#We still need to add HH:MM:SS for our Dates to be imported into MySQL

df_pol['Date'] = df_pol['Date'].dt.strftime('%Y-%m-%d %H:%M:%S')

df_pol

,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
Event ID,,,,,,,,
1,2002-07-02 00:00:00,Cairns,ship,commercial,port,"Trinity Inlet, Cairns Harbour",diesel,unknown
2,2002-07-03 00:00:00,Cairns,unknown,recreational,coastal waters,"Marina Mirage, Port Douglas",bilge,unknown
3,2002-07-12 00:00:00,Gladstone,ship,unknown,coastal waters,SE of Double Island Point,bilge,unknown
4,2002-07-22 00:00:00,Brisbane,unknown,unknown,coastal waters,"Town Reach, Brisbane River",diesel,unknown
5,2002-08-01 00:00:00,Gold Coast,ship,recreational,coastal waters,Canal area - Benowa,diesel,unknown
...,...,...,...,...,...,...,...,...
1094,2020-06-15 00:00:00,Gladstone,ship,recreational,port limits,"Gladstone Marina, Gladstone",other,<90 litres
1095,2020-06-16 00:00:00,Townsville,ship,commercial,port limits,"Breakwater Terminal, Ross Creek, Townsville",sewage,> 100 litres
1096,2020-06-16 00:00:00,Brisbane,ship,commercial,port limits,"Fisherman Island 3 berth, Port of Brisbane",hydraulic oil,100- 200ml


In [21]:
#Checking content of each column - detect missing/incorrect/duplicates values
df_region = df_pol["Region"].unique()
df_region


array(['Cairns', 'Gladstone', 'Brisbane', 'Gold Coast', 'Mackay',
       'Townsville', 'At Sea', 'Gladstone ', 'Brisbale', 'Brisbane ',
       'Cairns ', 'Hay Point', 'Bundaberg', 'Townsville '], dtype=object)

In [22]:
df_source = df_pol["Source"].unique()
df_source

array(['ship', 'unknown', 'land', 'aircraft', 'helicopter',
       'truck in water', nan, 'coral'], dtype=object)

In [23]:
df_ship = df_pol["Ship Type"].unique()
df_ship

array(['commercial', 'recreational', 'unknown', 'fishing', nan, 'trading',
       'tanker', 'defence', 'oil tanker', 'helicopter', 'customs',
       'n/a - museum piece', 'recreation', 'land', 'naval',
       'bulk carrier', 'trading ship', 'sailling vessel', 'navy', 'na',
       'rec', 'comm', 'fish'], dtype=object)

In [24]:
df_area = df_pol["Area"].unique()
df_area

array(['port', 'coastal waters', 'offshore', 'gbr', 'coastal', 'gbrmp',
       'territorial sea', 'port limits', 'marina', 'inland waters'],
      dtype=object)

In [25]:
df_pollutant = df_pol["Pollutant"].unique()
df_pollutant

array(['diesel', 'bilge', 'cotton seed residue', 'hfo', 'hydraulic oil',
       'garbage', 'paint', 'tar', 'tallo', 'petrol', 'sheen', 'hydraulic',
       'fuel oil', 'crude', 'avgas', 'paint dust', 'chemical', 'unknown',
       'other', 'fatty substance', 'lubricating oil', 'other - lube ',
       'other - lube', 'other - hyd', 'other - gearbox',
       'other - hydraulic', 'lube oil', 'coke', 'petroleum coke',
       'hydraulic ', 'discoloured water', 'diesel ', 'soot',
       'discoloured water ', 'oil', 'lubricating', 'engine oil',
       'light oil', 'vegetable', 'tallow ', ' hydraulic ', 'lubricating ',
       'engine ', 'coolant', ' petrol', 'heating ',
       'carbon from diesel engine', 'hydraulic fluid', nan,
       'palm acid oil', 'sodium hypochlorite', 'sulphur',
       'sewage & grey water', 'ground food waste 1890 kg', 'cooking oil',
       'chemical (aflco)', 'lanolin', 'other (hydraulic oil)',
       'other (calcined petroleum coke)', 'other (lube oil)',
       'other 

In [26]:
df_volume = df_pol["Estimated Litres"].unique()
df_volume

array(['unknown', '20', '50', '5', '15', '6', '2', '40', '60', '100',
       '500', '300', nan, '10', '2000', '180', '1000', '3000', '200',
       '30', '75', '1', '25', '3', '700', '7', '0.2', '7000', '0.5', '77',
       '4', '270,000', '35', '8', '120', '600', '400', '2500', '1500',
       '0.3', '0.1', '14000', '30kg', '250', '150', '> 10,000', '250mil',
       'sheen', '70', '11', '1.5', '2.5', '12', '<2l', '2-3l', '500x6m',
       '15l', '<1l', '100-200l', '200m x 10m', '95m x 25m', '20-30l',
       '1l', '100mx100m', '20l', '200mx4m', '100m x 50m', '200l',
       '44g drum', '2x2m', '4x5m', '600x150m', '100x3m', '10l', '650x2m',
       '30l', '200-400l', '600x20m', '2l', '10-15l', '0.5l', '20mx30m ',
       '200m', '4l', '200mx 50m', '50-60m', '1m x 10m', '0.5 x 5m', '50m',
       '0.5m x 3m', '50l', '1200l', '1000l', '< 20l', '100l', '40-50l',
       '< 5l', '7-10 m2', '50m x 5m', '6-8l', '20mx  5m', '3l', '32l ',
       '15m  wide ', '<5l', 'algal bloom', '3000m x 2000m', '100m

In [27]:
#Filling N/A

In [28]:
#Replacing Values

In [29]:
# Region #
df_pol['Region'] = df_pol['Region'].replace('Brisbale', 'Brisbane')

#Remove extra-space
df_pol["Region"] = df_pol["Region"].str.rstrip()

In [30]:
df_region = df_pol["Region"].unique()
df_region

array(['Cairns', 'Gladstone', 'Brisbane', 'Gold Coast', 'Mackay',
       'Townsville', 'At Sea', 'Hay Point', 'Bundaberg'], dtype=object)

In [31]:
# Source #

# Fill na data with "Unknown" - unless 'Ship Type' column shows they should be ship

# Find the rows with empty values in the 'Source' column
mask = df_pol['Source'].isnull()

# Filter the dataframe using the mask and check if Ship Type value is notnull, not unknown, not n/a
update_source_to_ship = df_pol.loc[mask & (df_pol['Ship Type'].notnull()) & (df_pol['Ship Type'] != 'unknown') & (df_pol['Ship Type'] != 'na') & (df_pol['Ship Type'] != 'n/a')]

#print('Source -> ship',update_source_to_ship)
#2 returned events have Ship Type = recreational; their Source value should be filled with ship

update_source_to_unknown = df_pol.loc[mask & ((df_pol['Ship Type'].isnull()) | (df_pol['Ship Type'] == 'unknown') | (df_pol['Ship Type'] == 'na') | (df_pol['Ship Type'] == 'n/a'))]
#print('Source -> unknown',update_source_to_unknown)
#5 other results should have Ship Type NAN, their Source should be filled as unknownbe filled with 


In [32]:
#Fill empty values in 'Source' with ship or unknown depending on conditions set and checked above

df_pol.loc[mask & (df_pol['Ship Type'].notnull()) & (df_pol['Ship Type'] != 'unknown') & (df_pol['Ship Type'] != 'na') & (df_pol['Ship Type'] != 'n/a'), 'Source'] = 'ship'
df_pol.loc[mask & ((df_pol['Ship Type'].isnull()) | (df_pol['Ship Type'] == 'unknown') | (df_pol['Ship Type'] == 'na') | (df_pol['Ship Type'] == 'n/a')), 'Source'] = 'unknown'

In [33]:
#Check values other than ship, land and unknown

other_mask = df_pol.loc[(df_pol["Source"] != "ship") & (df_pol["Source"] != "unknown") & (df_pol["Source"] != "land")]
other_mask


,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
Event ID,,,,,,,,
145,2004-04-02 00:00:00,Brisbane,aircraft,helicopter,port,"Kedron Brook Floodway, Brisbane",avgas,unknown
172,2004-07-30 00:00:00,Brisbane,helicopter,NaN,coastal waters,"Helicopter Pontoon, South Bank Reach",avgas,unknown
516,2011-04-19 00:00:00,Townsville,truck in water,NaN,coastal waters,"Palm Island Jetty, Palm Island",diesel,unknown
1002,2019-05-27 00:00:00,Gladstone,coral,NaN,port limits,"Gladstone Marina, Gladstone",trichodesmium,200m


In [34]:
#Source for the 4 rows returned above ('aircraft, helipcopter, truck, coral) to be replaced with 'other'

df_pol.loc[(df_pol["Source"] != "ship") & (df_pol["Source"] != "unknown") & (df_pol["Source"] != "land"), "Source"] = "other"


In [35]:
# Replace "Source" with "other" rather than ship for the "n/a - museum piece" with "n/a" in rows where "Ship Type" is "n/a - museum piece"

#print(df_pol.loc[df_pol['Ship Type'] == 'n/a - museum piece'])

df_pol.loc[df_pol['Ship Type'] == 'n/a - museum piece', 'Source'] = 'other'

In [36]:
df_source = df_pol["Source"].unique()
df_source

#Source now reduced to 4 values

array(['ship', 'unknown', 'land', 'other'], dtype=object)

In [37]:
# Ship Type #

In [38]:
#Commercial - add :  trading - oil tanker - tanker - trading ship - comm - bulk carrier
df_pol.loc[(df_pol["Ship Type"]== 'trading'), "Ship Type"] = "commercial"
df_pol.loc[(df_pol["Ship Type"]== 'tanker'), "Ship Type"] = "commercial"
df_pol.loc[(df_pol["Ship Type"]== 'oil tanker'), "Ship Type"] = "commercial"
df_pol.loc[(df_pol["Ship Type"]== 'trading ship'), "Ship Type"] = "commercial"
df_pol.loc[(df_pol["Ship Type"]== 'comm'), "Ship Type"] = "commercial"
df_pol.loc[(df_pol["Ship Type"]== 'bulk carrier'), "Ship Type"] = "commercial"

In [39]:
#Recreational - add : recreation - rec - 'sailling vessel'
df_pol.loc[(df_pol["Ship Type"]== 'recreation'), "Ship Type"] = "recreational"
df_pol.loc[(df_pol["Ship Type"]== 'rec'), "Ship Type"] = "recreational"
df_pol.loc[(df_pol["Ship Type"]== 'sailling vessel'), "Ship Type"] = "recreational"

In [40]:
#Fishing - add fish
df_pol.loc[(df_pol["Ship Type"]== 'fish'), "Ship Type"] = "fishing"

In [41]:
#Government - add : defence - naval - navy - customs
df_pol.loc[(df_pol["Ship Type"]== 'defence'), "Ship Type"] = "government"
df_pol.loc[(df_pol["Ship Type"]== 'naval'), "Ship Type"] = "government"
df_pol.loc[(df_pol["Ship Type"]== 'navy'), "Ship Type"] = "government"
df_pol.loc[(df_pol["Ship Type"]== 'customs'), "Ship Type"] = "government"

In [42]:
#Empty / N/A / Other Ship types

# Empty Ship Type values -> 'unknown' if 'Source' = 'ship'

df_pol.loc[(df_pol['Ship Type'].isnull()) & (df_pol['Source'] == 'ship'), 'Ship Type'] = 'unknown'


# Fill empty Ship Type values with "n/a" if "Source" is "land" or "unknown" or "other"

df_pol.loc[(df_pol['Ship Type'].isnull()) & (df_pol['Source'].isin(['land', 'unknown', 'other'])), 'Ship Type'] = 'n/a'

# Add 'na' to 'n/a' 
df_pol.loc[(df_pol['Ship Type']== 'na'), 'Ship Type'] = 'n/a'



In [43]:
#Other (helicopter, land, n/a - museum piece) -> n/a

df_pol.loc[(df_pol["Ship Type"] != "commercial") 
& (df_pol["Ship Type"] != "recreational") 
& (df_pol["Ship Type"] != "fishing")
& (df_pol["Ship Type"] != "government") 
& (df_pol["Ship Type"] != "n/a")
& (df_pol["Ship Type"] != "unknown"), "Ship Type"] = 'n/a'

In [44]:
ship_type_counts = df_pol['Ship Type'].value_counts()
print(ship_type_counts)

commercial      351
unknown         243
n/a             237
recreational    177
fishing          76
government       14
Name: Ship Type, dtype: int64


In [45]:
# Area #

#no N/A, need to check 

#Port Limits - add : port  - marina
df_pol.loc[(df_pol["Area"]== 'port'), "Area"] = "port limits"
df_pol.loc[(df_pol["Area"]== 'marina'), "Area"] = "port limits"

#Coastal Waters - add : coastal - offshore - territorial sea
df_pol.loc[(df_pol["Area"]== 'coastal'), "Area"] = "coastal waters"
df_pol.loc[(df_pol["Area"]== 'offshore'), "Area"] = "coastal waters"
df_pol.loc[(df_pol["Area"]== 'territorial sea'), "Area"] = "coastal waters"

#Great barrier reef: gbr - gbrmp (35 events)
df_pol.loc[(df_pol["Area"]== 'gbr'), "Area"] = "great barrier reef"
df_pol.loc[(df_pol["Area"]== 'gbrmp'), "Area"] = "great barrier reef"


In [46]:
area_counts = df_pol['Area'].value_counts()
print(area_counts)

port limits           662
coastal waters        391
great barrier reef     35
inland waters          10
Name: Area, dtype: int64


In [47]:
df_pol.describe()

,Date,Region,Source,Ship Type,Area,Location,Pollutant,Estimated Litres
count,1096,1098,1098,1098,1098,1098,1096,1077
unique,981,9,4,6,4,866,103,152
top,2014-07-18 00:00:00,Brisbane,ship,commercial,port limits,"Marina Mirage, Port Douglas",diesel,unknown
freq,3,347,664,351,662,15,463,603


In [48]:
df_pol.isnull().sum()

Date                 2
Region               0
Source               0
Ship Type            0
Area                 0
Location             0
Pollutant            2
Estimated Litres    21
dtype: int64

In [49]:
# Location
# Data is not consistent, but irrelevant for analysis

In [50]:
# Pollutant #
# sheen, diesel, bilge, heavy fuel oil (HFO) Thinking about sort all to "Oil"


# Will use sql count function to highlight the most common pollutant: 
# - in Australia (all datasets)
# - groupby region
# - groupby Ship Type
# - groupby date ? 


In [51]:
pollutant_counts = df_pol['Pollutant'].value_counts()
print(pollutant_counts)

diesel                       463
sheen                        185
bilge                        101
other                         43
hfo                           34
                            ... 
carbon from diesel engine      1
heating                        1
 petrol                        1
coolant                        1
sewage                         1
Name: Pollutant, Length: 103, dtype: int64


In [52]:
#Pollutant#

# Regroup all values that contain diesel
df_pol['Pollutant'] = df_pol['Pollutant'].replace(to_replace=r'.*diesel.*', value='diesel', regex=True)

# Regroup all values that contain sheen
df_pol['Pollutant'] = df_pol['Pollutant'].replace(to_replace=r'.*sheen.*', value='sheen', regex=True)

# Regroup all values that contain bilge
df_pol['Pollutant'] = df_pol['Pollutant'].replace(to_replace=r'.*bilge.*', value='bilge', regex=True)

# Regroup hfo / fuel oil / heavy fuel oil
df_pol.loc[(df_pol['Pollutant']== 'hfo'), 'Pollutant'] = 'fuel oil'
df_pol.loc[(df_pol['Pollutant']== 'heavy fuel oil'), 'Pollutant'] = 'fuel oil'

#Regroup all values that contain lub
df_pol['Pollutant'] = df_pol['Pollutant'].replace(to_replace=r'.*lub.*', value='lubricating oil', regex=True)

#Regroup all values that contain petrol
df_pol['Pollutant'] = df_pol['Pollutant'].replace(to_replace=r'.*petrol.*', value='petroleum', regex=True)

#Regroup all values that contain hydraulic
df_pol['Pollutant'] = df_pol['Pollutant'].replace(to_replace=r'.*hydraulic.*', value='hydraulic', regex=True)

#Regroup other oil
df_pol['Pollutant'] = df_pol['Pollutant'].replace(to_replace=r'.*oil.*', value='other oil', regex=True)

#Regroup all other pollutants (occuring between 1 and 4 times each)

df_pol.loc[(df_pol["Pollutant"] != "diesel") 
& (df_pol["Pollutant"] != "sheen") 
& (df_pol["Pollutant"] != "bilge")
& (df_pol["Pollutant"] != "fuel oil") 
& (df_pol["Pollutant"] != "lubricating oil")
& (df_pol["Pollutant"] != "petroleum")
& (df_pol["Pollutant"] != "hydraulic")
& (df_pol["Pollutant"] != "other oil")
& (df_pol["Pollutant"] != "unknown"), "Pollutant"] = 'other'

#Now reduced from 103 to 8 values


In [53]:
pollutant_counts = df_pol['Pollutant'].value_counts()
print(pollutant_counts)

diesel       475
sheen        186
hydraulic    114
other        110
bilge        105
other oil     77
unknown       20
petroleum     11
Name: Pollutant, dtype: int64


In [54]:
# Estimated Litres #

#if "ml", divide the number before by 1000 and remove ml
#if finishes with digit or space followed by "l" or "litre" or "litres", remove "l" or "litre" or "litres" 
#if "<" or ">", remove "<" or ">"
#if includes "-" between 2 numbers, average number before and number after
#if includes "to" between 2 number, average number before and number after
#if includes "/" between 2 numbers, divide 1st number by 2nd number
#if "," followed by "000", remove ","
#if "cubic metre", remove "cubic metre" and multiply number * 1000
#if "." between 2 numbers, convert string to float
#if "algal bloom" or "oily rags", copy value in the "Pollutant" column if the value in the Pollutant column is empty or "other"
#if "+l", remove "+l"
#if "mil", multiply number by 1000000
#if "sheen", delete
#if ends with "g" or "kg", replace with N/A (this is not a volume)
#if only 1 "x" and 1 or several m, replace with N/A (this is an area, not a volume)
#if ends with "m" or "m2" and "m" is the only alphabetic character, replace with N/A

#

In [62]:
def clean_volume(value):
    value = str(value)

    
    # Extract numbers and decimal point
    #numbers = ""
    #for char in value:
    #    if char.isdigit() or char == "." or char in ("-","to","/","x"):
    #        numbers += char
            

    # Remove "litre" or "litres"
    value = value.replace("litre", "")
    value = value.replace("litres", "")
    
    # Remove any commas
    value = value.replace(",", "")

    # Check for "algal bloom", "oily rags" or "sheen"
    if value in ["algal bloom", "oily rags", "sheen"]:
        return np.nan
    
    # Ignore values that are weights by checking for "g" or "kg"
    if "g" in value or "kg" in value:
        return np.nan
    
    # Check for "x"
    if "x" in value:
        return np.nan
    
    # Remove "<" or ">"
    if ">" in value or "<" in value:
        value = value.replace("<", "").replace(">", "")

    # Remove "+l"
    if "+l" in value:
        value = value.replace("+l", "")
        
    # Convert string to float
    try:
        volume = float(value)
    except ValueError:
        return np.nan
    
    # Convert "mil" to "million"
    if "mil" in value:
        value = value.replace("mil", "")
        volume *= 1000000

    # Convert "ml"
    if "ml" in value:
        value = value.replace("ml", "")
        volume /= 1000
 
    # Remove "l" measure unit
    if value.endswith("l"):
        value = value[:-1]
        volume = float(value)

    # Convert "cubic metre"
    if "cubic metre" in value:
        value = value.replace("cubic metre", "")
        volume *= 1000
    
    # Check for "-" and average both numbers
    if "-" in value:
        values = value.split("-")
        if all(val.isdigit() for val in values):
            volume = (float(values[0]) + float(values[1])) / 2
        else:
            return np.nan

    # Check for "to" and average both numbers
    if "to" in value:
        values = value.split("to")
        if all(val.isdigit() for val in values):
            volume = (float(values[0]) + float(values[1])) / 2
        else:
            return np.nan

    # Check for "/"
    if "/" in value:
        values = value.split("/")
        if all(val.isdigit() for val in values):
            volume = float(values[0]) / float(values[1])
        else:
            return np.nan
        
    
    # Check for "-" or "to" or "/" or "x"
    #if "-" in value or "to" in value or "/" in value or "x" in value:
    #    return np.nan
    
    # Check for "-" and average both numbers
    #if "-" in value:
        #values = value.split("-")
        #try:
        #volume = (float(values[0]) + float(values[1])) / 2
        #except ValueError:
    #    return np.nan

    # Check for "to" and average both numbers
    #if "to" in value:
        #values = value.split("to")
        #try:
        #volume = (float(values[0]) + float(values[1])) / 2
        #except ValueError:
    #    return np.nan

    # Check for "/"
    #if "/" in value:
        #values = value.split("/")
        #try:
        #volume = float(values[0]) / float(values[1])
        #except ValueError:
    #    return np.nan
       
    return volume

In [63]:
# Apply the function to the "Estimated Litres" column
df_pol['Estimated Litres'] = df_pol['Estimated Litres'].apply(clean_volume)

# Convert the "Estimated Litres" column to numeric values
#df_pol['Estimated Litres'] = pd.to_numeric(df_pol['Estimated Litres'], errors='coerce')

# Print cleaned DataFrame
print(df_pol)

                         Date      Region   Source     Ship Type  \
Event ID                                                           
1         2002-07-02 00:00:00      Cairns     ship    commercial   
2         2002-07-03 00:00:00      Cairns  unknown  recreational   
3         2002-07-12 00:00:00   Gladstone     ship       unknown   
4         2002-07-22 00:00:00    Brisbane  unknown       unknown   
5         2002-08-01 00:00:00  Gold Coast     ship  recreational   
...                       ...         ...      ...           ...   
1094      2020-06-15 00:00:00   Gladstone     ship  recreational   
1095      2020-06-16 00:00:00  Townsville     ship    commercial   
1096      2020-06-16 00:00:00    Brisbane     ship    commercial   
1097      2020-06-23 00:00:00      Cairns  unknown       unknown   
1098      2020-06-29 00:00:00      Cairns     land  recreational   

                    Area                                       Location  \
Event ID                                

In [64]:
df_pol.isnull().sum()

Date                  2
Region                0
Source                0
Ship Type             0
Area                  0
Location              0
Pollutant             0
Estimated Litres    744
dtype: int64

In [65]:
df_pol.to_csv("pollution-cleaning-vol-wip.csv", encoding="utf8")

In [59]:
#Command SQL

#Select Region. Pollutant. COUNT(*)
#FROM pollution_3
#GROUP BY Region. Pollutant
#ORDER BY COUNT(*) DESC;

In [60]:
# - groupby region

#SELECT Region, Pollutant, COUNT(*)
#FROM `pollution-test_3_datetime_ascii`
#WHERE Region = "name region" -- input any regions
#GROUP BY Region, Pollutant
#ORDER BY COUNT(*) DESC;

In [1]:
# -- identify pollutant for a specific year

#SELECT Region,Pollutant, Count(Pollutant)
#FROM `pollution-test_3_datetime_ascii`
#WHERE Date like "2005%" AND Pollutant = "diesel"
#GROUP BY Region, Pollutant
#ORDER BY Count(Pollutant) DESC;

In [2]:
df_region = df_pol["Region"].unique()

NameError: name 'df_region' is not defined